# Part 5 - Test Data Preprocessing

Import Module

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

/Users/tengfone/.pyenv/versions/3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Pathings

In [2]:
TF_train_pathing = "/Users/tengfone/Desktop/ML_Project/data/EN(2)/train"
TF_test_pathing = "/Users/tengfone/Desktop/ML_Project/data/test_data/test.in"

Loading Files

In [3]:
def open_train_file(file_path):
    with open(file_path) as f_lines:
        all_datas = f_lines.read().splitlines()
        all_datas[:] = [x for x in all_datas if x]

    word_tags = list()
    output_word_tags = list()
    for i in all_datas:
        i = i.split(' ')
        word_tags.append(i)
        
    for i in word_tags:
        if(len(i) == 3):
            i = [i[0]+i[1],i[2]]
            output_word_tags.append(i)
        else:
            output_word_tags.append(i)
    return output_word_tags

In [4]:
def open_test_file(file_path):
    with open(file_path) as f_lines:
        all_datas = f_lines.read().splitlines()

    output = list()
    para = 0

    for i in all_datas:
        if i != "":
            output.append([i, para])
        else:
            para += 1

    return output

Convert to DF

In [5]:
def to_train_df(word_tags):
    df = pd.DataFrame(word_tags, columns=['Text', 'Labels'])
    return df

In [6]:
def to_test_df(list_words):
    df = pd.DataFrame(list_words, columns=['Text', 'Paragraph'])
    return df

Emission Part a

In [7]:
def emission(train_data):
    # Total Number of Labels (DF)
    df_num_labels = train_data['Labels'].value_counts().rename_axis('Labels').reset_index(name='CountY')

    # Total Number of y -> x
    df_emission = train_data.groupby(["Text","Labels"]).size().reset_index()
    df_emission.columns = ["Text", "Labels", "CountY->X"]

    # e(x|y)
    emission_output = pd.merge(df_emission,df_num_labels, on = "Labels")
    emission_output["Emission"] = emission_output["CountY->X"]/emission_output["CountY"]

    return emission_output

Emission Part b (Fix with UNK)

In [8]:
def replace_unk(word,list_of_words):
    if(word not in list_of_words):
        return "#UNK#"
    else:
        return word

In [9]:
def emission_fix(train_data,test_data,k):
    list_of_words = train_data['Text'].to_list()
    test_data['Text'] = test_data['Text'].apply(lambda x: replace_unk(x,list_of_words))
  
  # Total Number of Labels (DF)
    df_num_labels = train_data['Labels'].value_counts().rename_axis('Labels').reset_index(name='CountY')

  # Total Number of y -> x
    df_emission = train_data.groupby(["Text","Labels"]).size().reset_index()
    df_emission.columns = ["Text", "Labels", "CountY->X"]

  # e(x|y)
    emission_output = pd.merge(df_emission,df_num_labels, on = "Labels")

  # Unique Labelsoutlist.append(output)
    unique_labels = emission_output.Labels.unique().tolist()
    print(emission_output)

    for i in range(len(unique_labels)):
        each_label = unique_labels[i]
        print(f"Running at {each_label}")
        count_y = emission_output[emission_output['Labels']==each_label]['CountY'].values[0]
        temp_list = list()
        for index, row in test_data.iterrows():
            word = row['Text']
            if (word == "#UNK#"):
                output = k/ (count_y + k)
                temp_list.append(output)
            else:
                count_yx = 0
                a = emission_output[(emission_output['Text'] == word)]
                a2 = a[(a['Labels'] == each_label)]
                # a = df.query('Text==@word and Labels==@label')
                if (a2.size==0):
                    output = 0
                    temp_list.append(output)
                else:
                    a2 = a2['CountY->X'].values[0]
                    count_yx = a2
                    output = count_yx/(count_y + k)
                    temp_list.append(output)
        test_data[each_label] = temp_list
                    
    return test_data

Get new Fixed Emission

In [10]:
k = 0.5
# Training
train_data = open_train_file(TF_train_pathing)
train_data_df = to_train_df(train_data)
emitted = emission(train_data_df)

# Testing
test_data = open_test_file(TF_test_pathing)

test_data_df = to_test_df(test_data)
emitted_fix = emission_fix(train_data_df,test_data_df,k)

         Text  Labels  CountY->X  CountY
0           !       O         15   23872
1           #       O          1   23872
2           $       O          2   23872
3           &       O          3   23872
4           '       O         19   23872
...       ...     ...        ...     ...
25046  before  I-SBAR          1      48
25047      if  I-SBAR         15      48
25048    that  I-SBAR          9      48
25049  though  I-SBAR         15      48
25050    wine   B-UCP          1       1

[25051 rows x 4 columns]
Running at O
Running at B-ADJP
Running at B-NP
Running at I-NP
Running at I-ADJP
Running at I-ADVP
Running at B-PP
Running at I-VP
Running at B-VP
Running at B-ADVP
Running at I-INTJ
Running at I-UCP
Running at B-LST
Running at B-SBAR
Running at B-INTJ
Running at B-CONJP
Running at B-PRT
Running at I-PP
Running at I-CONJP
Running at I-SBAR
Running at B-UCP


In [11]:
emitted_fix

,Text,Paragraph,O,B-ADJP,B-NP,I-NP,I-ADJP,I-ADVP,B-PP,I-VP,...,I-UCP,B-LST,B-SBAR,B-INTJ,B-CONJP,B-PRT,I-PP,I-CONJP,I-SBAR,B-UCP
0,Estimated,0,0.000000,0.000000,0.000042,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0
1,and,0,0.085580,0.000571,0.000148,0.016669,0.029591,0.060523,0.002610,0.004528,...,0.222222,0.0,0.000000,0.0,0.0,0.0000,0.049217,0.0,0.0,0.0
2,actual,0,0.000000,0.000000,0.000148,0.000128,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0
3,results,0,0.000000,0.000000,0.000127,0.000989,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0
4,involving,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25708,year,1093,0.000000,0.000000,0.000063,0.007199,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0
25709,and,1093,0.085580,0.000571,0.000148,0.016669,0.029591,0.060523,0.002610,0.004528,...,0.222222,0.0,0.000000,0.0,0.0,0.0000,0.049217,0.0,0.0,0.0
25710,in,1093,0.000000,0.000000,0.000021,0.000037,0.000000,0.000000,0.155649,0.000000,...,0.000000,0.0,0.004738,0.0,0.0,0.0619,0.026846,0.0,0.0,0.0
25711,1990,1093,0.000042,0.000000,0.000444,0.000348,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0


Write to csv file

In [12]:
emitted_fix.to_csv(r'TEST.csv')